# 🏆 Tydzień 2, Dzień 6: Projekt - 10-K Financial Analyzer

To finałowy projekt tego tygodnia. Zbudujemy zaawansowany system RAG, który potrafi analizować skomplikowane raporty roczne (Form 10-K).

**Kluczowe funkcje:**
1.  **Hybrid Ingestion:** Traktujemy tekst i tabele osobno. Tabele zamieniamy na Markdown, aby LLM je zrozumiał.
2.  **Unified Vector Store:** Wszystko (tekst i tabele) trafia do jednej bazy ChromaDB.
3.  **Smart Retrieval:** Gemini 2.5 Flash otrzymuje kontekst i precyzyjnie odpowiada na pytania typu "Jaki był wzrost przychodów r/r?".

---
### 🛠️ 1. Instalacja

Używamy pełnego zestawu z tego tygodnia.

In [ ]:
! uv add langchain langchain-community langchain-google-genai chromadb pdfplumber pandas tabulate reportlab python-dotenv

### 📝 2. Generowanie Raportu 10-K (Tekst + Tabele)

Stworzymy bogaty plik PDF.
*   **Strona 1:** Opis biznesu i Ryzyka (Tekst).
*   **Strona 2:** Rachunek Zysków i Strat (Skomplikowana Tabela).

In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import os

# Ścieżki
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../../"))
reports_dir = os.path.join(project_root, "data", "reports")
os.makedirs(reports_dir, exist_ok=True)

pdf_path = os.path.join(reports_dir, "tech_corp_10k.pdf")

def create_10k_report(filename):
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    
    # --- STRONA 1: TEKST (Ryzyka) ---
    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, height - 50, "FORM 10-K: TechCorp Innovations Inc.")
    
    c.setFont("Helvetica", 12)
    text_lines = [
        "ITEM 1A. RISK FACTORS",
        "",
        "1. Supply Chain Dependence:",
        "We rely heavily on a single supplier for our quantum processors.",
        "Any disruption in Helium-3 supply could halt our operations for 6 months.",
        "",
        "2. Regulatory Changes:",
        "New AI safety regulations in the EU (AI Act) may require us to redesign",
        "our core algorithms, costing approximately $50M in compliance fees.",
        "",
        "3. Competition:",
        "Competitors like Google and IBM are investing heavily in Qubits.",
        "If they achieve Quantum Supremacy before us, our market share may drop by 40%."
    ]
    
    y = height - 100
    for line in text_lines:
        c.drawString(50, y, line)
        y -= 20
    
    c.showPage() # Nowa strona
    
    # --- STRONA 2: TABELA (Finanse) ---
    height = A4[1] # Reset wysokości
    c.setFont("Helvetica-Bold", 14)
    c.drawString(50, height - 50, "ITEM 8. FINANCIAL STATEMENTS")
    
    # Rysujemy tabelę "spacjami" (trudną dla parserów)
    y = height - 100
    c.setFont("Helvetica", 11)
    
    # Nagłówki
    c.drawString(50, y, "Metric (in millions USD)")
    c.drawString(300, y, "2023")
    c.drawString(400, y, "2024")
    y -= 20
    c.line(50, y+15, 450, y+15)
    
    rows = [
        ("Total Revenue", "100.5", "150.2"),
        ("Cost of Goods Sold", "40.0", "60.0"),
        ("Gross Profit", "60.5", "90.2"),
        ("Operating Expenses", "50.0", "85.0"),
        ("Net Income", "10.5", "5.2"), # Spadek zysku mimo wzrostu przychodu!
        ("Earnings Per Share", "2.10", "1.04")
    ]
    
    for row in rows:
        c.drawString(50, y, row[0])
        c.drawString(300, y, row[1])
        c.drawString(400, y, row[2])
        y -= 20
        
    c.save()
    print(f"✅ Raport 10-K wygenerowany: {filename}")

create_10k_report(pdf_path)

✅ Raport 10-K wygenerowany: c:\Users\takze\OneDrive\Pulpit\project\financial-ai-engineering\data\reports\tech_corp_10k.pdf


### ⚙️ 3. Klasa `DocumentIngestor` (ETL)

To jest nasz **Parser**. Jego zadaniem jest:
1.  Przejść przez każdą stronę PDF.
2.  Spróbować znaleźć tabelę (`pdfplumber`).
    *   Jeśli jest -> Zamień na Markdown -> Zrób z tego `Document`.
3.  Jeśli nie ma tabeli -> Weź tekst -> Potnij go (Chunking) -> Zrób z tego `Document`.

Dzięki temu w bazie będziemy mieli i tekst, i sformatowane tabele.

In [2]:
import pdfplumber
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

class DocumentIngestor:
    def __init__(self, chunk_size=500, chunk_overlap=50):
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )

    def process_pdf(self, pdf_path):
        """Zwraca listę dokumentów LangChain (tekst + tabele)."""
        final_docs = []
        
        print(f"🔄 Przetwarzanie: {pdf_path}")
        
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                # 1. Próba wyciągnięcia tabeli
                # Używamy strategii 'text' z wczoraj (działa najlepiej na PDF bez linii)
                settings = {"vertical_strategy": "text", "horizontal_strategy": "text"}
                table_data = page.extract_table(table_settings=settings)
                
                if table_data:
                    print(f"   📄 Strona {i+1}: Wykryto tabelę.")
                    # Zamiana na Markdown
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])
                    df = df.fillna("0").replace(',', '', regex=True) # Cleaning
                    md_table = df.to_markdown(index=False)
                    
                    # Dodajemy jako dokument. Ważne: dodajemy opis co to jest.
                    doc_content = f"FINANCIAL TABLE (Page {i+1}):\n{md_table}"
                    final_docs.append(Document(page_content=doc_content, metadata={"source": pdf_path, "page": i+1, "type": "table"}))
                
                else:
                    print(f"   📄 Strona {i+1}: Tekst.")
                    # 2. Jeśli nie ma tabeli, bierzemy tekst
                    text = page.extract_text()
                    if text:
                        chunks = self.splitter.split_text(text)
                        for chunk in chunks:
                            final_docs.append(Document(page_content=chunk, metadata={"source": pdf_path, "page": i+1, "type": "text"}))
                            
        return final_docs

# Testujemy Ingestora
ingestor = DocumentIngestor()
documents = ingestor.process_pdf(pdf_path)

print(f"\n✅ Uzyskano {len(documents)} fragmentów wiedzy.")
print("Przykładowy fragment (Tabela):")
print(documents[-1].page_content) # Powinna być tabela z końca PDF

🔄 Przetwarzanie: c:\Users\takze\OneDrive\Pulpit\project\financial-ai-engineering\data\reports\tech_corp_10k.pdf
   📄 Strona 1: Wykryto tabelę.
   📄 Strona 2: Wykryto tabelę.

✅ Uzyskano 2 fragmentów wiedzy.
Przykładowy fragment (Tabela):
FINANCIAL TABLE (Page 2):
| ITEM 8. FINANCIAL STATEMENTS   |       |       |
|:-------------------------------|:------|:------|
|                                |       |       |
| Metric (in millions USD)       | 2023  | 2024  |
|                                |       |       |
| Total Revenue                  | 100.5 | 150.2 |
|                                |       |       |
| Cost of Goods Sold             | 40.0  | 60.0  |
|                                |       |       |
| Gross Profit                   | 60.5  | 90.2  |
|                                |       |       |
| Operating Expenses             | 50.0  | 85.0  |
|                                |       |       |
| Net Income                     | 10.5  | 5.2   |
|                     

### 🧠 4. Klasa `FinancialRAG` (Mózg)

To jest silnik.
1.  Tworzy bazę ChromaDB z dokumentów od Ingestora.
2.  Używa **Gemini 2.5 Flash** do generowania odpowiedzi.

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma

# Ładowanie kluczy
load_dotenv(os.path.join(project_root, ".env"))

class FinancialRAG:
    def __init__(self, docs):
        print("⚙️ Inicjalizacja bazy wektorowej i modelu...")
        
        # 1. Embeddings & LLM
        self.embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
        
        # 2. Vector DB (w pamięci dla szybkości)
        self.db = Chroma.from_documents(
            documents=docs,
            embedding=self.embeddings,
            collection_name="10k_final_project"
        )
        
        self.retriever = self.db.as_retriever(search_kwargs={"k": 4})

    def analyze(self, query):
        print(f"\n🔎 Analiza pytania: '{query}'")
        
        # 1. Retrieval
        relevant_docs = self.retriever.invoke(query)
        
        # Budowanie kontekstu z cytatami
        context_str = ""
        for d in relevant_docs:
            source_type = d.metadata.get("type", "unknown")
            page = d.metadata.get("page", "?")
            context_str += f"\n--- [Źródło: Strona {page} ({source_type})] ---\n{d.page_content}\n"
            
        # 2. Prompting
        system_prompt = """
        Jesteś ekspertem analizy raportów 10-K.
        Masz dostęp do fragmentów tekstu oraz tabel finansowych w formacie Markdown.
        
        Twoim zadaniem jest odpowiedzieć na pytanie użytkownika PRECYZYJNIE.
        - Jeśli pytanie dotyczy liczb, wykonaj obliczenia (np. różnica między latami).
        - Jeśli pytanie dotyczy ryzyk, wypunktuj je.
        - ZAWSZE powołuj się na dane z kontekstu.
        """
        
        full_prompt = f"{system_prompt}\n\nKONTEKST:\n{context_str}\n\nPYTANIE UŻYTKOWNIKA:\n{query}"
        
        # 3. Generation
        response = self.llm.invoke(full_prompt)
        
        return response.content, context_str

# Uruchomienie Silnika
bot = FinancialRAG(documents)
print("✅ Bot gotowy do pracy.")

⚙️ Inicjalizacja bazy wektorowej i modelu...
✅ Bot gotowy do pracy.


### 🎮 5. Testy Końcowe

Zadamy botowi 3 rodzaje pytań:
1.  **O Ryzyka** (wymaga czytania tekstu ze strony 1).
2.  **O Liczby** (wymaga odczytania Tabeli ze strony 2).
3.  **O Wnioski** (wymaga połączenia faktów: Przychód wzrósł, ale Zysk spadł).

In [ ]:
# TEST 1: Ryzyka
answer, source = bot.analyze("Jakie są główne ryzyka związane z łańcuchem dostaw?")
print(f"🤖 ODPOWIEDŹ:\n{answer}")

# TEST 2: Konkretna liczba z tabeli
answer, source = bot.analyze("Jaki był zysk netto (Net Income) w 2024 roku?")
print(f"🤖 ODPOWIEDŹ:\n{answer}")

# TEST 3: Analiza (Wnioskowanie)
answer, source = bot.analyze("Dlaczego zysk netto spadł w 2024 mimo wzrostu przychodów? Przeanalizuj koszty.")
print(f"🤖 ODPOWIEDŹ:\n{answer}")

## 🧠 Zadanie Domowe (Capstone Tygodnia 2)

Zauważ, że w tabeli wiersz "Operating Expenses" wzrósł z 50.0 do 85.0. W tekście o ryzykach (strona 1) jest wzmianka o "compliance fees" ($50M) w związku z AI Act.

**Zadanie:**
1. Zmodyfikuj PDF (Komórka 4), dodając w sekcji Ryzyka (na stronie 1) zdanie: *"We anticipate Operating Expenses to rise significantly due to new AI compliance costs."*
2. Uruchom cały notatnik od nowa.
3. Zadaj botowi pytanie: *"Co mogło być przyczyną wzrostu kosztów operacyjnych?"*.
4. Sprawdź, czy bot potrafi połączyć **Tabelę ze strony 2** (wzrost kosztów) z **Tekstem ze strony 1** (przyczyna wzrostu). To jest tzw. **Multi-hop Reasoning**.

In [ ]:
# --- ROZWIĄZANIE ZADANIA DOMOWEGO ---

# 1. MODYFIKACJA PDF (Dodajemy "most" logiczny)
def create_updated_10k_report(filename):
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    
    # --- STRONA 1: TEKST (Zmieniony) ---
    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, height - 50, "FORM 10-K: TechCorp Innovations Inc.")
    
    c.setFont("Helvetica", 12)
    text_lines = [
        "ITEM 1A. RISK FACTORS",
        "",
        "1. Supply Chain Dependence:",
        "We rely heavily on a single supplier for our quantum processors.",
        "Any disruption in Helium-3 supply could halt our operations for 6 months.",
        "",
        "2. Regulatory Changes:",
        "New AI safety regulations in the EU (AI Act) may require us to redesign",
        "our core algorithms, costing approximately $50M in compliance fees.",
        # --- DODANE ZDANIE (Klucz do zadania) ---
        "We anticipate Operating Expenses to rise significantly due to new AI compliance costs.",
        # ----------------------------------------
        "",
        "3. Competition:",
        "Competitors like Google and IBM are investing heavily in Qubits.",
        "If they achieve Quantum Supremacy before us, our market share may drop by 40%."
    ]
    
    y = height - 100
    for line in text_lines:
        c.drawString(50, y, line)
        y -= 20
    
    c.showPage()
    
    # --- STRONA 2: TABELA (Bez zmian) ---
    height = A4[1]
    c.setFont("Helvetica-Bold", 14)
    c.drawString(50, height - 50, "ITEM 8. FINANCIAL STATEMENTS")
    
    y = height - 100
    c.setFont("Helvetica", 11)
    c.drawString(50, y, "Metric (in millions USD)")
    c.drawString(300, y, "2023")
    c.drawString(400, y, "2024")
    y -= 20
    c.line(50, y+15, 450, y+15)
    
    rows = [
        ("Total Revenue", "100.5", "150.2"),
        ("Cost of Goods Sold", "40.0", "60.0"),
        ("Gross Profit", "60.5", "90.2"),
        ("Operating Expenses", "50.0", "85.0"), # Wzrost z 50 do 85
        ("Net Income", "10.5", "5.2"),
        ("Earnings Per Share", "2.10", "1.04")
    ]
    
    for row in rows:
        c.drawString(50, y, row[0])
        c.drawString(300, y, row[1])
        c.drawString(400, y, row[2])
        y -= 20
        
    c.save()
    print(f"✅ Zaktualizowany raport wygenerowany: {filename}")

# 2. URUCHOMIENIE PEŁNEGO PROCESU
# Generujemy nowy plik
create_updated_10k_report(pdf_path)

# Ponowne przetworzenie (Ingestion)
print("\n🔄 Ponowne ładowanie dokumentu...")
ingestor = DocumentIngestor()
new_docs = ingestor.process_pdf(pdf_path)

# Nowy Bot z nową wiedzą
print("🧠 Aktualizacja wiedzy bota...")
bot_updated = FinancialRAG(new_docs)

# 3. TEST (Multi-hop Reasoning)
query = "Co mogło być przyczyną wzrostu kosztów operacyjnych (Operating Expenses)?"
answer, source = bot_updated.analyze(query)

print(f"🤖 ODPOWIEDŹ AI (Multi-hop):\n{answer}")

# Weryfikacja
print("\n--- Analiza poprawności ---")
print("AI powinno zauważyć w tabeli wzrost kosztów (50 -> 85) i powiązać to")
print("z tekstem o 'AI compliance costs' ($50M) na stronie 1.")